In [1]:
from __future__ import print_function
import sys
import numpy as np
import pandas as pd
pd.options.display.max_rows = 5
import matplotlib.pyplot as plt
import gvar as gv
import fkfpi_lib as flib
print("python version:", sys.version)
print("pandas version:", pd.__version__)
print("numpy  version:", np.__version__)
print("gvar   version:", gv.__version__)

python version: 2.7.14 (default, Jan  6 2018, 12:15:00) 
[GCC 4.2.1 Compatible Apple LLVM 9.0.0 (clang-900.0.39.2)]
pandas version: 0.20.3
numpy  version: 1.13.1
gvar   version: 8.2.2


In [2]:
switches = dict()
switches['ensemble'] = [
    'a15m310','a12m310','a09m310',
    'a15m220','a12m220','a09m220','a12m220S','a12m220L',
    'a15m130','a12m130']
switches['ansatz'] = dict()
switches['ansatz']['FV'] = True # True turns on NLO FV correction
switches['ansatz']['type'] = 'MA' # Type of fit: 'xpt' or 'MA'
switches['ansatz']['a2dm'] = 'avg' # avg or individual
switches['scale'] = 'PP' # PP, PK, KK, LamChi = 4 * pi * sqrt(FA * FB)

In [3]:
# Set priors for Xpt
priors = dict()
priors['L5'] = gv.gvar(0.0, 1.0)
priors['s2'] = gv.gvar(0.0, 1.0)
# Physical parameters from PDG
phys_p = dict()
# http://pdg.lbl.gov/2016/tables/rpp2016-tab-mesons-light.pdf
phys_p['mpi'] = gv.gvar(134.8, 0.3) #FLAG 2017 (16)
phys_p['mka'] = gv.gvar(494.2 , 0.3) #FLAG 2017 (16)
phys_p['F0'] = gv.gvar(80,20) #FLAG use of F0 in SU(2) correction for FK/Fpi
phys_p['meta'] = gv.gvar(547.862,0.017) #PDG
# http://pdg.lbl.gov/2015/reviews/rpp2015-rev-pseudoscalar-meson-decay-cons.pdf
phys_p['Fpi'] = gv.gvar(130.2/np.sqrt(2), 1.7/np.sqrt(2)) #PDG fpi+ eq(16)
phys_p['Fka'] = gv.gvar(155.6/np.sqrt(2), 0.4/np.sqrt(2)) #PDG fK++ eq(16)
if switches['scale'] == 'PK':
    phys_p['Lchi'] = 4*np.pi*np.sqrt(phys_p['Fpi']*phys_p['Fka'])
elif switches['scale'] == 'PP':
    phys_p['Lchi'] = 4*np.pi*phys_p['Fpi']
elif switches['scale'] == 'KK':
    phys_p['Lchi'] = 4*np.pi*phys_p['Fka']
# FLAG FK/Fpi
flag_FKFpi = gv.gvar(1.1933, 0.0029)

In [4]:
data = pd.read_csv('./github_fkfpi_v1.csv')
hisq_params = pd.read_csv('./hisq_params.csv')
mixed_params = pd.read_csv('./github_mixed_v1.csv')

In [5]:
fmt_data = flib.format_data(switches,data,mixed_params,hisq_params,priors)
#print(fmt_data)

In [11]:
fit = flib.fit_data(switches,fmt_data,phys_p)
print(fit.format(maxline=True,pstyle='m'))

Least Square Fit:
  chi2/dof [dof] = 5.6 [10]    Q = 2.4e-08    logGBF = 12.128

Parameters:
         a2dm 0     0.03328 (58)       [  0.03373 (59) ]  
              1     0.01071 (35)       [  0.01077 (36) ]  
              2     0.00195 (14)       [  0.00197 (14) ]  
              3     0.03046 (60)       [  0.03034 (62) ]  
              4     0.01095 (27)       [  0.01084 (28) ]  
              5    0.001522 (92)       [ 0.001504 (93) ]  
              6     0.01057 (30)       [  0.01055 (30) ]  
              7     0.01081 (30)       [  0.01077 (30) ]  
              8     0.03230 (81)       [  0.03252 (82) ]  
              9     0.01147 (32)       [  0.01136 (32) ]  
          mka 0     0.40416 (23)       [  0.40460 (25) ]  *
              1     0.32409 (19)       [  0.32416 (21) ]  
              2     0.24089 (12)       [  0.24110 (14) ]  *
              3     0.38699 (20)       [  0.38691 (21) ]  
              4     0.31014 (16)       [  0.31003 (17) ]  
              5    0

In [24]:
for i,e in enumerate(switches['ensemble']):
    print('%8s %s %s' %(e,str(fmt_data['y'][i]),str(fit.fcn(fit.x,fit.p)[i])))

 a15m310 1.0969(14) 1.09124(71)
 a12m310 1.1003(15) 1.09921(65)
 a09m310 1.1093(16) 1.10476(64)
 a15m220 1.1255(18) 1.1270(10)
 a12m220 1.1302(21) 1.13434(84)
 a09m220 1.1433(17) 1.14640(77)
a12m220S 1.1325(27) 1.13437(90)
a12m220L 1.1276(28) 1.13483(89)
 a15m130 1.1680(41) 1.1621(18)
 a12m130 1.1593(35) 1.1692(12)


In [7]:
print(flib.fkfpi_phys(phys_p,fit))

prediction from LQCD
FK / Fpi =  1.1884(18)
FK+/Fpi+ =  1.1860(18)
prediction from LQCD + meta_PDG
FK / Fpi =  1.1858(19)
FK+/Fpi+ =  1.1834(19)
None
